<a href="https://colab.research.google.com/github/abecode/630_django_example/blob/master/first_try_mbert_all_at_once.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## downloading data and model


In [ ]:
# download the data
# right now it's commented out to prevent accidental load to the semeval server
# !curl  https://propaganda.math.unipd.it/semeval2023task3/data/semeval2023task3bundle-v1.tgz  | tar -xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.3M  100 13.3M    0     0  5357k      0  0:00:02  0:00:02 --:--:-- 5355k


In [ ]:
# get bert model
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip multi_cased_L-12_H-768_A-12.zip

--2022-10-30 21:10:58--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.13.80, 172.253.62.128, 172.217.0.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.13.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   237MB/s    in 2.7s    

2022-10-30 21:11:00 (237 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L

## imports

In [46]:
import os
import re


In [ ]:
!!pip install -q tf-models-official==2.4.0

In [57]:
import tensorflow as tf
from official.modeling import tf_utils
from official import nlp
from official.nlp import bert
# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

## data loading

In [ ]:
data_dir = 'data'

In [ ]:
en_train_articles_subtask1 = {}
for fname in os.scandir(os.path.join(data_dir, "en", "train-articles-subtask-1")):
  if fname.is_file() and fname.name.endswith("txt"):
    article_id = re.search(r'article(\d+)\.txt$', fname.path).group(1)
    #print(article_id)
    en_train_articles_subtask1[article_id] = fname.path




In [ ]:
en_train_labels_subtask1 = {}
with open(os.path.join(data_dir, "en", "train-labels-subtask-1.txt")) as f:
  for line in f:
    article_id, genre = line.strip().split("\t")
    en_train_labels_subtask1[article_id] = genre


In [ ]:
# symmetric difference between articles and labels
# should be empty
assert not set(en_train_labels_subtask1) ^ set(en_train_articles_subtask1)

In [ ]:
train_articles_subtask1 = {}
langs = ["en", "fr", "it", "po", "ru"]
for lang in langs:
  for fname in os.scandir(os.path.join(data_dir, lang, "train-articles-subtask-1")):
    if fname.is_file() and fname.name.endswith("txt"):
      article_id = re.search(r'article(\d+)\.txt$', fname.path).group(1)
      #print(article_id)
      train_articles_subtask1[article_id] = fname.path

train_labels_subtask1 = {}
for lang in langs:
  with open(os.path.join(data_dir, lang, "train-labels-subtask-1.txt")) as f:
    for line in f:
      article_id, genre = line.strip().split("\t")
      train_labels_subtask1[article_id] = genre

# symmetric difference between articles and labels
# should be empty
assert not set(train_labels_subtask1) ^ set(train_articles_subtask1)

## tokenizer

In [59]:
bert_model_dir = "multi_cased_L-12_H-768_A-12"
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(bert_model_dir, "vocab.txt"),
     do_lower_case=True)

print("Vocab size:", len(tokenizer.vocab))

Vocab size: 119547


In [63]:
# test tokenizer
tokens = tokenizer.tokenize("Hello TensorFlow!")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
tokens = tokenizer.tokenize("subset(e,nightmare)")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

some_polish = open("data/po/train-articles-subtask-1/article251.txt").read().rstrip()
print(some_polish)
tokens = tokenizer.tokenize(some_polish)
print(tokens)
print(len(tokens))

['hell', '##o', 'ten', '##sor', '##flow', '!']
[61694, 10133, 11769, 28919, 105739, 106]
['sub', '##set', '(', 'e', ',', 'night', '##mare', ')']
[13987, 14488, 113, 173, 117, 16903, 34918, 114]
Nord Stream 2 największym problemem niemieckiej polityki

Gazociąg Nord Stream 2 i wiążąca się z nim zależność energetyczna od Rosji to największy problem niemieckiej polityki zagranicznej - pisze dziennik „Sueddeutsche Zeitung”, podkreślając, że Niemcy muszą uwolnić się od "od zgubnej zależności" od rosyjskiego gazu.

Taka zależność istniała już w czasie zimnej wojny, wówczas jednak argumentowano, że dostawca i klient potrzebują siebie nawzajem i powinni we własnym interesie oddzielać biznes od polityki – dodaje gazeta.

Niemcy potrzebują rosyjskiego gazu, a Rosja niemieckich pieniędzy, jednak „prezydent Rosji (Władimir Putin) myśli strategicznie” i kwestię dochodu odstawił na dalszy plan – zaznacza "SZ".

„Rosja od dawna realizuje strategię podziału Unii Europejskiej, izolowania poszczególnych

## 